# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [0]:
#import libraries
import numpy as np
import datetime

np.random.seed(42) 
import tensorflow as tf
tf.reset_default_graph()
import keras
from keras.utils.np_utils import to_categorical
from keras import backend as K
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dense, Dropout

In [0]:
#import CIFAR10 dataset
from keras.datasets import cifar10

In [0]:
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((50000, 32, 32, 3), (50000, 1), (10000, 32, 32, 3), (10000, 1))

In [0]:
#Flatten the Y dataset
y_train = y_train.flatten()
y_test = y_test.flatten()


In [0]:
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gt5 = x_train[y_train >= 5]
y_train_gt5 = y_train[y_train >= 5] - 5  # make classes start at 0 for
x_test_gt5 = x_test[y_test >= 5]         # np_utils.to_categorical
y_test_gt5 = y_test[y_test >= 5] - 5


In [0]:
print(x_train_lt5.shape)
print(y_train_lt5.shape)
print(x_test_lt5.shape)
print(y_test_lt5.shape)

(25000, 32, 32, 3)
(25000,)
(5000, 32, 32, 3)
(5000,)


In [0]:
print(x_train_gt5.shape)
print(y_train_gt5.shape)
print(x_test_gt5.shape)
print(y_test_gt5.shape)

(25000, 32, 32, 3)
(25000,)
(5000, 32, 32, 3)
(5000,)


In [0]:
now = datetime.datetime.now

batch_size = 128
nb_classes = 5
nb_epoch = 5

# input image dimensions
img_rows, img_cols = 32, 32
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = 2
# convolution kernel size
kernel_size = 3

input_shape=(32,32,3)

### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [0]:
y_train_class_lt5= keras.utils.to_categorical(y_train_lt5,nb_classes)
y_test_class_lt5 = keras.utils.to_categorical(y_test_lt5,nb_classes)

y_train_class_gt5 = keras.utils.to_categorical(y_train_gt5,nb_classes)
y_test_class_gt5 = keras.utils.to_categorical(y_test_gt5,nb_classes)

### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [0]:
x_train_lt5 = x_train_lt5.astype('float32') / 255
x_test_lt5 = x_test_lt5.astype('float32') / 255

In [0]:
x_train_lt5 = x_train_lt5.reshape(x_train_lt5.shape[0],32,32,3)
x_test_lt5 = x_test_lt5.reshape(x_test_lt5.shape[0],32,32,3)

In [0]:
cnn_1 = Sequential()
cnn_1.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(32, 32, 3), name='conv_1'))
cnn_1.add(MaxPooling2D(pool_size=(2, 2)))
cnn_1.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(32, 32, 3), name='conv_2'))
cnn_1.add(MaxPooling2D(pool_size=(2, 2)))

cnn_1.add(Conv2D(128, kernel_size=(3, 3), activation='relu', input_shape=(32, 32, 3), name='conv_3'))
cnn_1.add(MaxPooling2D(pool_size=(2, 2)))

cnn_1.add(Dropout(0.25))

cnn_1.add(Flatten())
cnn_1.add(Dense(128, activation='relu', name='dense_1'))
cnn_1.add(Dense(5, activation='softmax', name='dense_2'))

cnn_1.summary()

cnn_1.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

# fits the model on batches with real-time data augmentation:
model_1 = cnn_1.fit(x_train_lt5, y_train_class_lt5, 
                     validation_data=(x_test_lt5, y_test_class_lt5),        
                     batch_size=32, epochs=5)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv2D)              (None, 30, 30, 64)        1792      
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 15, 15, 64)        0         
_________________________________________________________________
conv_2 (Conv2D)              (None, 13, 13, 64)        36928     
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 6, 6, 64)          0         
_________________________________________________________________
conv_3 (Conv2D)              (None, 4, 4, 128)         73856     
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 2, 2, 128)         0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 2, 2, 128)         0         
__________

In [0]:
from keras.preprocessing.image import ImageDataGenerator


# fits the model on batches with real-time data augmentation:
model_1 = cnn_1.fit(x_train_lt5, y_train_class_lt5, 
                     validation_data=(x_test_lt5, y_test_class_lt5),        
                     batch_size=32, epochs=5)

Epoch 1/20
781/781 [==============================] - 21s 27ms/step - loss: 0.8427 - acc: 0.6755 - val_loss: 0.5992 - val_acc: 0.7776
Epoch 2/20
781/781 [==============================] - 21s 27ms/step - loss: 0.8015 - acc: 0.6926 - val_loss: 0.6370 - val_acc: 0.7634
Epoch 3/20
781/781 [==============================] - 21s 27ms/step - loss: 0.7812 - acc: 0.7019 - val_loss: 0.6288 - val_acc: 0.7710
Epoch 4/20
781/781 [==============================] - 21s 27ms/step - loss: 0.7512 - acc: 0.7160 - val_loss: 0.5539 - val_acc: 0.7910
Epoch 5/20
781/781 [==============================] - 21s 27ms/step - loss: 0.7337 - acc: 0.7216 - val_loss: 0.5620 - val_acc: 0.7848
Epoch 6/20
781/781 [==============================] - 20s 26ms/step - loss: 0.7248 - acc: 0.7233 - val_loss: 0.5800 - val_acc: 0.7900
Epoch 7/20
781/781 [==============================] - 20s 26ms/step - loss: 0.7124 - acc: 0.7317 - val_loss: 0.5953 - val_acc: 0.7836
Epoch 8/20
781/781 [==============================] - 20s 26ms

In [0]:
score = cnn_1.evaluate(x_test_lt5, y_test_class_lt5)

print('Test loss:', score[0])
print('Test accuracy:', score[1])
print(score)

5000/5000 [==============================] - 1s 134us/step
Test loss: 0.5068833152294159
Test accuracy: 0.81
[0.5068833152294159, 0.81]


### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [0]:
#Freezing layers in the model which don't have 'dense' in their name
for layer in cnn_1.layers:
  if('dense' not in layer.name): #prefix detection to freeze layers which does not have dense
    #Freezing a layer
    layer.trainable = False

#Module to print colourful statements
from termcolor import colored

#Check which layers have been frozen 
for layer in cnn_1.layers:
  print (colored(layer.name, 'blue'))
  print (colored(layer.trainable, 'red'))

conv_1
False
max_pooling2d_17
False
conv_2
False
max_pooling2d_18
False
conv_3
False
max_pooling2d_19
False
dropout_9
False
flatten_8
False
dense_1
True
dense_2
True


### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [0]:
x_train_gt5 = x_train_gt5.astype('float32') / 255
x_test_gt5 = x_test_gt5.astype('float32') / 255

x_train_gt5 = x_train_gt5.reshape(x_train_gt5.shape[0],32,32,3)
x_test_gt5 = x_test_gt5.reshape(x_test_gt5.shape[0],32,32,3)

In [0]:

# fits the model on batches with real-time data augmentation:
model_1 = cnn_1.fit(x_train_gt5, y_train_class_gt5, 
                     validation_data=(x_test_gt5, y_test_class_gt5),        
                     batch_size=32, epochs=5)

Train on 25000 samples, validate on 5000 samples
Epoch 1/5
  512/25000 [..............................] - ETA: 8s - loss: 2.4909 - acc: 0.1934

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


25000/25000 [==============================] - 9s 372us/step - loss: 0.8307 - acc: 0.6816 - val_loss: 0.5570 - val_acc: 0.7894
Epoch 2/5
25000/25000 [==============================] - 9s 373us/step - loss: 0.5178 - acc: 0.8129 - val_loss: 0.4529 - val_acc: 0.8376
Epoch 3/5
25000/25000 [==============================] - 9s 372us/step - loss: 0.4362 - acc: 0.8401 - val_loss: 0.4144 - val_acc: 0.8518
Epoch 4/5
25000/25000 [==============================] - 9s 374us/step - loss: 0.3797 - acc: 0.8622 - val_loss: 0.3986 - val_acc: 0.8546
Epoch 5/5
25000/25000 [==============================] - 9s 373us/step - loss: 0.3375 - acc: 0.8793 - val_loss: 0.3583 - val_acc: 0.8664


In [0]:
score = cnn_1.evaluate(x_test_gt5, y_test_class_gt5)

print('Test loss:', score[0])
print('Test accuracy:', score[1])
print(score)

5000/5000 [==============================] - 1s 131us/step
Test loss: 0.35829650840759275
Test accuracy: 0.8664
[0.35829650840759275, 0.8664]


## Sentiment analysis <br> 

The objective of the second problem is to perform Sentiment analysis from the tweets data collected from the users targeted at various mobile devices.
Based on the tweet posted by a user (text), we will classify if the sentiment of the user targeted at a particular mobile device is positive or not.

### 6. Read the dataset (tweets.csv) and drop the NA's while reading the dataset

In [0]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

import warnings
warnings.filterwarnings('ignore')

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
df_tweets = pd.read_csv('/content/gdrive/My Drive/Greatlearning_AIML/Greatlakes/Residency_Labs_Solution/External/tweets.csv', encoding = "ISO-8859-1").dropna()



In [124]:
df_tweets.shape

(3291, 3)

In [125]:
df_tweets.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [126]:
df_tweets.describe()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
count,3291,3291,3291
unique,3282,9,4
top,RT @mention Marissa Mayer: Google Will Connect...,iPad,Positive emotion
freq,3,946,2672


In [127]:
df_tweets.shape

(3291, 3)

### Consider only rows having Positive emotion and Negative emotion and remove other rows from the dataframe.

In [128]:
df_tweets['is_there_an_emotion_directed_at_a_brand_or_product'].value_counts()

Positive emotion                      2672
Negative emotion                       519
No emotion toward brand or product      91
I can't tell                             9
Name: is_there_an_emotion_directed_at_a_brand_or_product, dtype: int64

In [0]:
df_tweets = df_tweets[(df_tweets['is_there_an_emotion_directed_at_a_brand_or_product'] == 'Positive emotion') | (df_tweets['is_there_an_emotion_directed_at_a_brand_or_product'] == 'Negative emotion')]

In [130]:
df_tweets.shape

(3191, 3)

### 7. Represent text as numerical data using `CountVectorizer` and get the document term frequency matrix

#### Use `vect` as the variable name for initialising CountVectorizer.

In [132]:
X = df_tweets['tweet_text']
Y = df_tweets['is_there_an_emotion_directed_at_a_brand_or_product']
X.shape

(3191,)

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()

### 8. Find number of different words in vocabulary

In [0]:
dtm = vect.fit_transform(X)

In [135]:
dtm.shape

(3191, 5648)

In [136]:
words = vect.get_feature_names()
print(words)

['000', '02', '03', '08', '10', '100', '100s', '100tc', '101', '106', '10am', '10k', '10mins', '10pm', '10x', '11', '11ntc', '11th', '12', '12b', '12th', '13', '130', '14', '1406', '1413', '1415', '15', '150', '1500', '150m', '157', '15am', '15k', '16162', '16gb', '16mins', '17', '188', '1986', '1990style', '1m', '1of', '1pm', '1st', '20', '200', '2010', '2011', '2012', '20s', '21', '22', '23', '24', '25', '250k', '25th', '2am', '2day', '2honor', '2moro', '2nd', '2nite', '2s', '2yrs', '30', '300', '3000', '30a', '30am', '30p', '30pm', '32', '32gb', '35', '36', '37', '3d', '3g', '3gs', '3k', '3rd', '3x', '40', '400', '40min', '41', '45', '45am', '47', '48', '4android', '4chan', '4g', '4nqv92l', '4sq', '4sq3', '4square', '50', '54', '55', '58', '59', '59p', '59pm', '5pm', '5th', '60', '64g', '64gb', '64gig', '64mb', '65', '6hours', '6th', '70', '75', '7th', '80', '800', '80s', '81', '82', '89', '8am', '8p', '8pm', '8th', '90', '900', '911tweets', '95', '96', '967', '97', '98', '99', '99å

#### Tip: To see all available functions for an Object use dir

In [137]:
dir(vect)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_char_ngrams',
 '_char_wb_ngrams',
 '_check_stop_words_consistency',
 '_check_vocabulary',
 '_count_vocab',
 '_get_param_names',
 '_get_tags',
 '_limit_features',
 '_more_tags',
 '_sort_features',
 '_stop_words_id',
 '_validate_custom_analyzer',
 '_validate_params',
 '_validate_vocabulary',
 '_white_spaces',
 '_word_ngrams',
 'analyzer',
 'binary',
 'build_analyzer',
 'build_preprocessor',
 'build_tokenizer',
 'decode',
 'decode_error',
 'dtype',
 'encoding',
 'fit',
 'fit_transform',
 'fixed_vocabulary_',
 'get_feature_names',
 'get_params',
 'get_stop_words',
 'input',
 'inverse_transf

### Find out how many Positive and Negative emotions are there.

Hint: Use value_counts on that column

In [138]:
pd.value_counts(df_tweets['is_there_an_emotion_directed_at_a_brand_or_product'])

Positive emotion    2672
Negative emotion     519
Name: is_there_an_emotion_directed_at_a_brand_or_product, dtype: int64

###  Change the labels for Positive and Negative emotions as 1 and 0 respectively and store in a different column in the same dataframe named 'label'

Hint: use map on that column and give labels

In [0]:
df_tweets['label'] = df_tweets.is_there_an_emotion_directed_at_a_brand_or_product.map({'Positive emotion':1, 'Negative emotion':0})

### 9. Define the feature set (independent variable or X) to be `text` column and `labels` as target (or dependent variable)  and divide into train and test datasets

In [0]:
x = df_tweets.tweet_text
y = df_tweets['label']

In [141]:
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=2)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(2393,)
(798,)
(2393,)
(798,)


## 10. **Predicting the sentiment:**


### Use Naive Bayes and Logistic Regression and their accuracy scores for predicting the sentiment of the given text

In [0]:
# Navives Bayes
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [0]:
vect = CountVectorizer(ngram_range=(1, 1))
X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)

In [148]:
nb.fit(X_train_dtm, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [0]:
y_pred = nb.predict(X_test_dtm)

In [151]:
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred)

0.8671679197994987

In [0]:
# LogisticRegression
reg = LogisticRegression()

In [154]:
# train the model using X_train_dtm
reg.fit(X_train_dtm, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
# make class predictions for X_test_dtm
y_pred_class = reg.predict(X_test_dtm)

In [157]:
# calculate accuracy of class predictions
metrics.accuracy_score(y_test, y_pred_class)

0.868421052631579

In [158]:
# print the confusion matrix
metrics.confusion_matrix(y_test, y_pred_class)

array([[ 53,  77],
       [ 28, 640]])

## 11. Create a function called `tokenize_predict` which can take count vectorizer object as input and prints the accuracy for x (text) and y (labels)

In [0]:
def tokenize_test(vect):
    x_train_dtm = vect.fit_transform(X_train)
    print('Features: ', x_train_dtm.shape[1])
    x_test_dtm = vect.transform(X_test)
    nb = MultinomialNB()
    nb.fit(x_train_dtm, y_train)
    y_pred_class = nb.predict(x_test_dtm)
    print('Accuracy: ', metrics.accuracy_score(y_test, y_pred_class))

### Create a count vectorizer function which includes n_grams = 1,2  and pass it to tokenize_predict function to print the accuracy score

In [172]:
# include 1-grams and 2-grams
vect = CountVectorizer(ngram_range=(1, 2))
tokenize_test(vect)

Features:  24650
Accuracy:  0.8759398496240601


### 12. Create a count vectorizer function with stopwords = 'english'  and pass it to tokenize_predict function to print the accuracy score

In [173]:
vect1 = CountVectorizer(stop_words='english')
tokenize_test(vect1)

Features:  4661
Accuracy:  0.8659147869674185


### 13. Create a count vectorizer function with stopwords = 'english' and max_features =300  and pass it to tokenize_predict function to print the accuracy score

In [174]:
vect2 = CountVectorizer(stop_words='english',max_features =300)
tokenize_test(vect2)

Features:  300
Accuracy:  0.8233082706766918


### 14. Create a count vectorizer function with n_grams = 1,2  and max_features = 15000  and pass it to tokenize_predict function to print the accuracy score

In [175]:
vect3 = CountVectorizer(ngram_range=(1, 2),max_features =15000)
tokenize_test(vect3)

Features:  15000
Accuracy:  0.8771929824561403


### 15. Create a count vectorizer function with n_grams = 1,2  and include terms that appear at least 2 times (min_df = 2)  and pass it to tokenize_predict function to print the accuracy score

In [176]:
vect4 = CountVectorizer(ngram_range=(1, 2),min_df = 2)
tokenize_test(vect4)

Features:  7925
Accuracy:  0.8796992481203008
